In [ ]:
"""
Created on Thu Oct 17 15:13 2024

Regrid the BedMachine to a lower resolution

@author: Clara Burgard
"""

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from pyproj import Transformer
import pandas as pd
import sys,os
import time
from tqdm.notebook import tqdm
import multimelt.plume_functions as pf
import cc3d

In [2]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


In [3]:
outputpath_BedMachine='/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'
inputpath_BedMachine='/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/'

SET ALL LAND TO LAND


In [4]:
BedMachine_orig = xr.open_dataset(inputpath_BedMachine+'BedMachineAntarctica-v3.nc')


In [5]:
mask_0_2_3 = BedMachine_orig['mask'].where(BedMachine_orig['mask'] != 4, 2).where(BedMachine_orig['mask'] != 1, 2)
mask_0_1_2 = mask_0_2_3.where(mask_0_2_3 != 3, 1).astype(float)
mask_0_1_2.attrs['flag_meanings'] = 'ocean floating_ice ice_free_land_and_grounded_ice'
mask_0_1_2.attrs['flag_values'] = '[0 1 2]'
mask_0_1_2.attrs['valid_range'] = '[0 2]'
mask_0_1_2.to_dataset(name='mask_0_1_2').to_netcdf(outputpath_BedMachine + 'BedMachine_v3_mask_012.nc')

SET LAND VOSTOK TO GROUNDED ICE

In [6]:
mask_0_1_2_3 = BedMachine_orig['mask'].where(BedMachine_orig['mask'] != 4, 2).astype(float)
mask_0_1_2_3.attrs['flag_meanings'] = 'ocean ice_free_land grounded_ice floating_ice'
mask_0_1_2_3.attrs['flag_values'] = '[0 1 2 3]'
mask_0_1_2_3.attrs['valid_range'] = '[0 3]'
mask_0_1_2_3.to_dataset(name='mask_0_1_2_3').to_netcdf(outputpath_BedMachine + 'BedMachine_v3_mask_0123.nc')

DOWNSCALE TO 2 KM

In [7]:
weights_filter = np.zeros((5,5)) + 1
weights_filter[0,:] = 0.5
weights_filter[:,0] = 0.5
weights_filter[4,:] = 0.5
weights_filter[:,4] = 0.5
weights_filter[0,4] = 0.25
weights_filter[4,0] = 0.25
weights_filter[4,4] = 0.25
weights_filter[0,0] = 0.25

weights_da = xr.DataArray(data=weights_filter,dims=['y0','x0'])

In [8]:
mask_0_1_2_3 = xr.open_dataset(outputpath_BedMachine + 'BedMachine_v3_mask_0123.nc')['mask_0_1_2_3']
grid_2km = xr.open_dataset(inputpath_BedMachine + 'Mask_Iceshelf_IMBIE2_v2.nc')

In [9]:
mask_ocean = (mask_0_1_2_3 == 0).astype(float)
mask_rock = (mask_0_1_2_3 == 1).astype(float)
mask_ground = (mask_0_1_2_3 == 2).astype(float)
mask_isf = (mask_0_1_2_3 == 3).astype(float)

In [10]:
corr_ocean = pf.xr_nd_corr_v2(mask_ocean, weights_filter).sel(x=grid_2km.x,y=grid_2km.y)
corr_rock = pf.xr_nd_corr_v2(mask_rock, weights_filter).sel(x=grid_2km.x,y=grid_2km.y)
corr_ground = pf.xr_nd_corr_v2(mask_ground, weights_filter).sel(x=grid_2km.x,y=grid_2km.y)
corr_isf = pf.xr_nd_corr_v2(mask_isf, weights_filter).sel(x=grid_2km.x,y=grid_2km.y)

compute concentrations

In [11]:
corr_ocean_norm = corr_ocean/16
corr_rock_norm = corr_rock/16
corr_ground_norm = corr_ground/16
corr_isf_norm = corr_isf/16

set everything where there is a bit of ice shelf to 1

In [12]:
ls_mask_01 = ((corr_ground_norm > 0) | (corr_rock_norm > 0)).astype(int)
ls_mask_02 = ls_mask_01.where(ls_mask_01 == 0, 2)
ls_mask_012 = ls_mask_02.where(corr_isf == 0, 1)

correct ice shelf points that are within grounded ice


In [13]:
dx = abs(ls_mask_012.x[1] - ls_mask_012.x[0]).values.astype(int)
dy = abs(ls_mask_012.y[1] - ls_mask_012.y[0]).values.astype(int)

In [14]:
isf_only_mask = ls_mask_012 == 1

connectivity = 4
threshold = 1

dusted = cc3d.dust(isf_only_mask.values.astype(np.int64), 
           threshold = threshold, 
           connectivity = connectivity, 
           in_place = False)

labels_out = cc3d.connected_components(dusted, 
                               connectivity = connectivity)

labelled = xr.DataArray(labels_out, 
                coords = {"y": ls_mask_012.y, "x": ls_mask_012.x}, 
                dims = ["y", "x"],
                name = "labels")

# filter that checks the point around
weights_filter = np.zeros((3,3))
weights_filter[0,1] = 1
weights_filter[1,0] = 1
weights_filter[1,2] = 1
weights_filter[2,1] = 1

weights_da = xr.DataArray(data=weights_filter,dims=['y0','x0'])

new_mask = ls_mask_012.copy()
new_corr_ground_norm = corr_ground_norm.copy()
new_corr_isf_norm = corr_isf_norm.copy()

for conn_label in tqdm(range(1,labels_out.max()+1)):
    dom_region = isf_only_mask.where(labelled == conn_label, drop=True)
    dom_bounds_plus1 = np.array([dom_region.x.min().values - dx,dom_region.x.max().values + dx,dom_region.y.min().values - dy,dom_region.y.max().values + dy]).astype(int)
    dom_plus1_mask = isf_only_mask.sel(x=range(dom_bounds_plus1[0],dom_bounds_plus1[1]+1,dx), y=range(dom_bounds_plus1[2],dom_bounds_plus1[3]+1,dy))
    corr = pf.xr_nd_corr_v2(dom_plus1_mask, weights_filter)
    only_contour = (corr ^ dom_plus1_mask)
    neighboring_pixels = ls_mask_012.where(only_contour).where(labelled != conn_label)
    if neighboring_pixels.min() > 0:
        print('There is no ocean, this cannot be an ice shelf! I am putting it to grounded ice!')
        new_mask = new_mask.where(labelled != conn_label, 2)
        new_corr_ground_norm = new_corr_ground_norm.where(labelled != conn_label, corr_isf_norm + corr_ground_norm)
        new_corr_isf_norm = new_corr_isf_norm.where(labelled != conn_label, 0)

  0%|          | 0/1076 [00:00<?, ?it/s]

There is no ocean, this cannot be an ice shelf! I am putting it to grounded ice!
There is no ocean, this cannot be an ice shelf! I am putting it to grounded ice!
There is no ocean, this cannot be an ice shelf! I am putting it to grounded ice!
There is no ocean, this cannot be an ice shelf! I am putting it to grounded ice!
There is no ocean, this cannot be an ice shelf! I am putting it to grounded ice!
There is no ocean, this cannot be an ice shelf! I am putting it to grounded ice!
There is no ocean, this cannot be an ice shelf! I am putting it to grounded ice!
There is no ocean, this cannot be an ice shelf! I am putting it to grounded ice!
There is no ocean, this cannot be an ice shelf! I am putting it to grounded ice!
There is no ocean, this cannot be an ice shelf! I am putting it to grounded ice!
There is no ocean, this cannot be an ice shelf! I am putting it to grounded ice!
There is no ocean, this cannot be an ice shelf! I am putting it to grounded ice!
There is no ocean, this cann

correct ocean points that are within ice

In [15]:
ocean_only_mask = ls_mask_012 == 0

connectivity = 4
threshold = 1

dusted = cc3d.dust(ocean_only_mask.values.astype(np.int64), 
           threshold = threshold, 
           connectivity = connectivity, 
           in_place = False)

labels_out = cc3d.connected_components(dusted, 
                               connectivity = connectivity)

labelled = xr.DataArray(labels_out, 
                coords = {"y": ls_mask_012.y, "x": ls_mask_012.x}, 
                dims = ["y", "x"],
                name = "labels")

# filter that checks the point around
weights_filter = np.zeros((3,3))
weights_filter[0,1] = 1
weights_filter[1,0] = 1
weights_filter[1,2] = 1
weights_filter[2,1] = 1

weights_da = xr.DataArray(data=weights_filter,dims=['y0','x0'])

new_corr_ocean_norm = corr_ocean_norm.copy()

for conn_label in tqdm(range(2,labels_out.max()+1)):
    dom_region = ocean_only_mask.where(labelled == conn_label, drop=True)
    dom_bounds_plus1 = np.array([dom_region.x.min().values - dx,dom_region.x.max().values + dx,dom_region.y.min().values - dy,dom_region.y.max().values + dy]).astype(int)
    dom_plus1_mask = ocean_only_mask.sel(x=range(dom_bounds_plus1[0],dom_bounds_plus1[1]+1,dx), y=range(dom_bounds_plus1[2],dom_bounds_plus1[3]+1,dy))
    corr = pf.xr_nd_corr_v2(dom_plus1_mask, weights_filter)
    only_contour = (corr ^ dom_plus1_mask)
    neighboring_pixels = ls_mask_012.where(only_contour).where(labelled != conn_label)
    if neighboring_pixels.min() > 0:
        print('There is no ocean around it, this cannot be an ocean point')
        #print(neighboring_pixels.min().values)
        new_mask = new_mask.where(labelled != conn_label, neighboring_pixels.min().values)
        if neighboring_pixels.min().values == 1:
            new_corr_isf_norm = new_corr_isf_norm.where(labelled != conn_label, corr_ocean_norm + corr_isf_norm)
        elif neighboring_pixels.min().values == 2:
            new_corr_ground_norm = new_corr_ground_norm.where(labelled != conn_label, corr_ocean_norm + corr_ground_norm)
        new_corr_ocean_norm = new_corr_ocean_norm.where(labelled != conn_label, 0)

  0%|          | 0/110 [00:00<?, ?it/s]

There is no ocean around it, this cannot be an ocean point
There is no ocean around it, this cannot be an ocean point
There is no ocean around it, this cannot be an ocean point
There is no ocean around it, this cannot be an ocean point
There is no ocean around it, this cannot be an ocean point
There is no ocean around it, this cannot be an ocean point
There is no ocean around it, this cannot be an ocean point
There is no ocean around it, this cannot be an ocean point
There is no ocean around it, this cannot be an ocean point
There is no ocean around it, this cannot be an ocean point
There is no ocean around it, this cannot be an ocean point
There is no ocean around it, this cannot be an ocean point
There is no ocean around it, this cannot be an ocean point
There is no ocean around it, this cannot be an ocean point
There is no ocean around it, this cannot be an ocean point
There is no ocean around it, this cannot be an ocean point
There is no ocean around it, this cannot be an ocean poi

Checking the sums


In [16]:
sum_norms = new_corr_ocean_norm + new_corr_isf_norm + new_corr_ground_norm + corr_rock_norm

In [18]:
sum_norms.min() # if min is not 1, it's not a drama, it's the contour! :)

<xarray.DataArray 'mask_0_1_2_3' ()>
array(1.)

Writing to netcdf

In [ ]:
all_masks = new_mask.to_dataset(name='mask_0_1_2')
all_masks['mask_0_1_2'].attrs['flag_meanings'] = 'only_ocean contains_floating_ice only_ice_free_land_and_grounded_ice'
all_masks['mask_0_1_2'].attrs['flag_values'] = '[0 1 2]'
#all_masks['mask_0_1_2'].attrs['valid_range'] = '[0 2]'

all_masks['ocean_conc'] = new_corr_ocean_norm
all_masks['ocean_conc'].attrs['long_name'] = 'Ocean concentration between 0 and 1'
all_masks['ocean_conc'].attrs['valid_range'] = '[0 1]'

all_masks['ground_conc'] = new_corr_ground_norm
all_masks['ground_conc'].attrs['long_name'] = 'Grounded ice concentration between 0 and 1'
#all_masks['ground_conc'].attrs['valid_range'] = '[0 1]'

all_masks['isf_conc'] = new_corr_isf_norm
all_masks['isf_conc'].attrs['long_name'] = 'Floating ice concentration between 0 and 1'
#all_masks['isf_conc'].attrs['valid_range'] = '[0 1]'

all_masks['icefree_conc'] = corr_rock_norm
all_masks['icefree_conc'].attrs['long_name'] = 'Ice-free ground concentration between 0 and 1'
#all_masks['icefree_conc'].attrs['valid_range'] = '[0 1]'

all_masks.to_netcdf(outputpath_BedMachine + 'BedMachine_v3_aggregated2km_masks_only.nc')

OTHER VARIABLES

In [19]:
all_masks = xr.open_dataset(outputpath_BedMachine + 'BedMachine_v3_aggregated2km_masks_only.nc')

In [20]:
weights_filter = np.zeros((5,5)) + 1
weights_filter[0,:] = 0.5
weights_filter[:,0] = 0.5
weights_filter[4,:] = 0.5
weights_filter[:,4] = 0.5
weights_filter[0,4] = 0.25
weights_filter[4,0] = 0.25
weights_filter[4,4] = 0.25
weights_filter[0,0] = 0.25

weights_da = xr.DataArray(data=weights_filter,dims=['y0','x0'])

In [21]:
for vvar in ['firn','surface','thickness','bed','errbed','geoid']: #
    print(vvar)
    mask_nonocean = (all_masks['ocean_conc'] == 0).astype(float)
    var_field = BedMachine_orig[vvar].sel(x=all_masks.x,y=all_masks.y).astype(float).where(mask_nonocean,0)
    var_4km = pf.xr_nd_corr_v2(var_field, weights_filter)
    mask_4km = pf.xr_nd_corr_v2(mask_nonocean, weights_filter)
    all_masks[vvar] = var_4km/mask_4km
    all_masks[vvar] = all_masks[vvar].where(np.isfinite(all_masks[vvar]),0)
    all_masks[vvar].attrs = BedMachine_orig[vvar].attrs
    

firn
surface
thickness
bed
errbed
geoid


In [ ]:
(new_corr_isf_norm > 0)

In [23]:
ice_draft = (all_masks['thickness'] - all_masks['surface'])


In [32]:
((new_corr_isf_norm > 0) & (ice_draft <= 0)).plot()

In [ ]:
all_masks.to_netcdf(outputpath_BedMachine + 'BedMachine_v3_aggregated2km_allvars.nc')

In [31]:
plt.figure()
ice_draft.where(ice_draft > 0).plot()

In [39]:
ice_draft.sel(x=slice(-2.515e6,-2.495e6), y=slice(1.28e6,1.34e6)).plot()

In [41]:
plt.figure()
new_corr_isf_norm.sel(x=slice(-2.515e6,-2.495e6), y=slice(1.28e6,1.34e6)).plot()

In [42]:
isf_conc_reg = new_corr_isf_norm.sel(x=slice(-2.515e6,-2.495e6), y=slice(1.28e6,1.34e6))
ice_draft_reg = ice_draft.sel(x=slice(-2.515e6,-2.495e6), y=slice(1.28e6,1.34e6))

In [44]:
isf_conc_reg.where(ice_draft_reg <= 0).plot()

In [47]:
plt.figure()
ice_draft_reg.where((ice_draft_reg <= 0) & (isf_conc_reg > 0)).plot()

In [70]:
ice_draft_BM = ((BedMachine_orig['thickness'] - BedMachine_orig['surface']))#.sel(x=slice(-2.515e6,-2.495e6), y=slice(1.34e6,1.28e6))

In [54]:
mask_BM = mask_0_1_2_3.sel(x=slice(-2.515e6,-2.495e6), y=slice(1.34e6,1.28e6))

In [55]:
mask_BM

<xarray.DataArray 'mask_0_1_2_3' (y: 121, x: 41)>
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])
Coordinates:
  * x        (x) int32 -2515000 -2514500 -2514000 ... -2496000 -2495500 -2495000
  * y        (y) int32 1340000 1339500 1339000 ... 1281000 1280500 1280000
Attributes:
    long_name:      mask
    grid_mapping:   mapping
    valid_range:    [0 3]
    flag_values:    [0 1 2 3]
    flag_meanings:  ocean ice_free_land grounded_ice floating_ice
    source:         Antarctic Digital Database (rock outcrop) and Jeremie Mou...

In [56]:
plt.figure()
mask_BM.plot()

In [64]:
plt.figure()
ice_draft_BM.where(ice_draft_BM >= 0).plot(vmax=10)

In [65]:
new_corr_isf_norm.sel(x=-2.5040e6,y=1.32e6)

<xarray.DataArray 'mask_0_1_2_3' ()>
array(0.25)
Coordinates:
    x        int32 -2504000
    y        int32 1320000

In [66]:
ice_draft.sel(x=-2.5040e6,y=1.32e6)

<xarray.DataArray ()>
array(-522.34973699)
Coordinates:
    x        int32 -2504000
    y        int32 1320000

In [71]:
mask_0_1_2_3.sel(x=slice(-2.5040e6-1000,-2.5040e6+1000),y=slice(1.32e6+1000,1.32e6-1000))

<xarray.DataArray 'mask_0_1_2_3' (y: 5, x: 5)>
array([[0., 0., 1., 3., 0.],
       [0., 1., 2., 3., 0.],
       [1., 2., 1., 3., 0.],
       [2., 2., 2., 3., 0.],
       [2., 2., 2., 3., 0.]])
Coordinates:
  * x        (x) int32 -2505000 -2504500 -2504000 -2503500 -2503000
  * y        (y) int32 1321000 1320500 1320000 1319500 1319000
Attributes:
    long_name:      mask
    grid_mapping:   mapping
    valid_range:    [0 3]
    flag_values:    [0 1 2 3]
    flag_meanings:  ocean ice_free_land grounded_ice floating_ice
    source:         Antarctic Digital Database (rock outcrop) and Jeremie Mou...

In [74]:
print(ice_draft_BM.sel(x=slice(-2.5040e6-1000,-2.5040e6+1000),y=slice(1.32e6+1000,1.32e6-1000)))

<xarray.DataArray (y: 5, x: 5)>
array([[   0.        ,    0.        ,  -84.2172    ,    1.        ,
           0.        ],
       [   0.        , -121.21034   , -109.572945  ,    1.        ,
           0.        ],
       [-250.7028    , -187.91447   , -160.39189   ,    0.99999994,
           0.        ],
       [-307.66766   , -241.62581   , -139.50444   ,    0.99999994,
           0.        ],
       [-341.19754   , -273.22134   , -202.62724   ,    1.        ,
           0.        ]], dtype=float32)
Coordinates:
  * x        (x) int32 -2505000 -2504500 -2504000 -2503500 -2503000
  * y        (y) int32 1321000 1320500 1320000 1319500 1319000


In [73]:
27.5/16

1.71875